# **NLP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. """

In [6]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

1. x1 = units of 1.5L COLI product
2. x2 = units of 1.5L PEPSA product
3. Revenue = 49000*x1 - x1^2 + 30*x2 - 2*x2^2
4. Cost per hectolitre of syrup = $150
5. S1 production line hourly capacity = 7100 units 
6. Working shifts = 8 hours/day, 5 days/week
7. Production constraint = x2 >= 0.5*x1.


In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [9]:
print(response2.choices[0].message.content)

The objective function to be maximized for the given problem is the profit function which is the difference between revenue and costs. Given revenue and cost functions, the objective function is:

Maximize Profit = (49000*X1 - X1^2 + 30*X2 - 2*X2^2) - 150*(X1/40 + X2/20)


In [10]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [11]:
print(response3.choices[0].message.content)

1. X1 + X2 <= 7100 * 8 * 5 
2. X2 >= 0.5 * X1
3. X1 >= 0 
4. X2 >= 0


### **Generate Pyomo Code**

In [12]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [13]:
Markdown(response4.choices[0].message.content)

Now let's model this problem using the Pyomo python library. 

First, we need to install the Pyomo library using pip, if it is not already installed. 

```python
pip install pyomo 
``` 

Then, we can solve the problem mentioned above by creating a Pyomo model. Here is the Pyomo model:

```python
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define variables X1 and X2
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# define the objective function
model.profit = Objective(expr = (49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2) - 150 * (model.X1/40 + model.X2/20), 
                         sense=maximize) 

# define constraints
# Production constraint corresponds to the capacity of the S1 production line, let's assume we have 4 working weeks
model.production_constraint = Constraint(expr = model.X1 + model.X2 <= 7100 * 8 * 5 * 4)

# Mix constraint corresponds to the necessity of maintaining the mixture ratio
model.mix_constraint = Constraint(expr =  model.X2 - 0.5 * model.X1 >= 0)

# invoke the solver
solver = SolverFactory('glpk')
solver.solve(model)

# print the results
print("X1 =", model.X1(), "X2 =", model.X2(), "Profit =", model.profit())
```
*Note: Please replace solver name 'glpk' with the solver available on your system for linear optimization problems.

This sample data is used in the constraint, which refers to the maximum production capacity allowable over a four-week period given an eight-hour shift each day for five days a week.

### **Run the code Generated by GPT4**

In [14]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.4 MB/s eta 0:00:00


In [15]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [16]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define variables X1 and X2
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# define the objective function
model.profit = Objective(expr = (49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2) - 150 * (model.X1/40 + model.X2/20),
                         sense=maximize)

# define constraints
# Production constraint corresponds to the capacity of the S1 production line, let's assume we have 4 working weeks
model.production_constraint = Constraint(expr = model.X1 + model.X2 <= 7100 * 8 * 5 * 4)

# Mix constraint corresponds to the necessity of maintaining the mixture ratio
model.mix_constraint = Constraint(expr =  model.X2 - 0.5 * model.X1 >= 0)

# invoke the solver
solver = SolverFactory('knitro')
solver.solve(model)

# print the results
print("X1 =", model.X1(), "X2 =", model.X2(), "Profit =", model.profit())

X1 = 16335.833342968803 X2 = 8167.916680998579 Profit = 400289175.7310366


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [19]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define variables X1 and X2
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)

# define the objective function
model.profit = Objective(expr = (49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2) - 150 * (model.X1/40 + model.X2/20),
                         sense=maximize)

# define constraints
# Production constraint corresponds to the capacity of the S1 production line, let's assume we have 4 working weeks
model.production_constraint = Constraint(expr = model.X1 + model.X2 <= 7100 * 8 * 5)

# Mix constraint corresponds to the necessity of maintaining the mixture ratio
model.mix_constraint = Constraint(expr =  model.X2 - 0.5 * model.X1 >= 0)

# invoke the solver
solver = SolverFactory('knitro')
solver.solve(model)

# print the results
print("X1 =", model.X1(), "X2 =", model.X2(), "Profit =", model.profit())

X1 = 16335.833333842567 X2 = 8167.916666951235 Profit = 400289176.04068875
